In [1]:
import pandas as pd
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

Using TensorFlow backend.


In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
#import boto3
#from boto3.s3.transfer import TransferConfig

## 1. Import Embeddings (X) and Labels (Y)

### 1.1 Embeddings

In [3]:
now = time.time()
embeddings = pickle.load(open( "embeddings6", "rb" ))
print(time.time()-now)

13.979804515838623


In [6]:
'''
now = time.time()
embeddings = {}
session = boto3.session.Session(region_name='us-east-1')
s3client = session.client('s3')
for i in ['embeddings1']:
          #,'embeddings2','embeddings3','embeddings4','embeddings5']:
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
print(time.time()-now)
'''

1210.0585293769836


In [4]:
embeddings_np = np.array(list(embeddings.items()))
embeddings_list = None

In [5]:
embeddings_np[:,1].shape

(2726,)

In [6]:
HADM_ID_LIST = embeddings_np[:,0]
embeddings = embeddings_np[:,1]
embeddings_np = None

In [7]:
len(embeddings), embeddings[0].shape

(2726, (26, 128, 768))

In [8]:
arr = np.array([embeddings[i].shape[0] for i in range(0,len(embeddings))])
for i in range(5, 80, 5):
    print(str(i)+': '+str(len(arr[arr<i])/len(embeddings)))

5: 0.008070432868672046
10: 0.03815113719735877
15: 0.1107850330154072
20: 0.26705796038151136
25: 0.4383712399119589
30: 0.5832721936903889
35: 0.7076302274394718
40: 0.8121790168745414
45: 0.8815113719735876
50: 0.9269992663242846
55: 0.9526779163609684
60: 0.9699192956713133
65: 0.983125458547322
70: 0.9889948642699926
75: 0.9944974321349963


#### 1.1.1 Zero pad and truncate

In [9]:
embeddings_truncated = None
embeddings_truncated = [np.float16(np.concatenate(i[:30])) for i in embeddings]

In [10]:
30*128

3840

In [11]:
embeddings_padded = []
for i in embeddings_truncated:
    pad_len = 3840 - len(i)
    if pad_len:
        embeddings_padded.append(np.append(i, np.zeros(pad_len*768).reshape(pad_len, 768), axis=0))
    else:
        embeddings_padded.append(i)
embeddings_padded = np.array(embeddings_padded)

In [13]:
embeddings_padded.shape

(2726, 3840, 768)

In [14]:
del(embeddings)
del(embeddings_truncated)
del(embeddings_np)
del(embeddings_list)

In [15]:
who

Activation	 Concatenate	 Dense	 Dropout	 Embedding	 Flatten	 GlobalMaxPooling1D	 HADM_ID_LIST	 Input	 
K	 LSTM	 Model	 Reshape	 Sequential	 Tokenizer	 arr	 cPickle	 embeddings_padded	 
i	 io	 now	 np	 one_hot	 pad_len	 pad_sequences	 pd	 pickle	 
plt	 time	 train_test_split	 


### 1.2 Labels - Top 100 Dx code and 18 category

In [16]:
CHAPTER_PIVOT_DF = pd.read_csv('s3://w210-mimic/data/chapter_label.csv', sep=',', header = 0)\
                    .astype({'HADM_ID': 'str'}).set_index('HADM_ID')

In [17]:
CHAPTER_PIVOT_DF.head()

,001_139,140_239,240_279,280_289,290_319,320_389,390_459,460_519,520_579,580_629,630_679__740_759__760_779,680_709,710_739,780_799,800_999,E_V
HADM_ID,,,,,,,,,,,,,,,,
100001,0,0,1,0,0,1,1,0,1,1,0,1,0,0,0,1
100003,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0
100006,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,1
100007,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0
100009,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1


In [18]:
HADM_ID_DF = None
HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])

In [19]:
CHAPTER_PIVOT_DF = HADM_ID_DF.set_index('HADM_ID')\
            .join(CHAPTER_PIVOT_DF, how='left')

In [20]:
HADM_ID_LIST[:5]

array(['116775', '194415', '113885', '156454', '148449'], dtype=object)

In [22]:
CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF)

In [24]:
CHAPTER_PIVOT_NP.shape

(2726, 16)

## 2. Split test and training

### 2.1 Top-100 ICD9

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(embeddings_padded, DIAGNOSIS_PIVOT_NP[:,18:], test_size=0.1, random_state=42)

### 2.2 18 ICD category as label

In [ ]:
#X_train, X_test, y_train1, y_test1 = train_test_split(embeddings_padded, DIAGNOSIS_PIVOT_NP[:,:18], test_size=0.1, random_state=42)

### 2.3 Both top-100 ICD9 and category as labels

In [25]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(embeddings_padded, CHAPTER_PIVOT_NP, test_size=0.15, random_state=42)

In [26]:
X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape

((2317, 3840, 768), (409, 3840, 768), (2317, 16), (409, 16))

## 3. Define LSTM pipeline

### 3.1 LSTM 1

In [27]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [32]:
deep_inputs = Input(shape=(3840,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

In [33]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1])

In [34]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3840, 768)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               459264    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
Total params: 461,328
Trainable params: 461,328
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history1 = model.fit(X_train2, y_train2, epochs=3, verbose=1, validation_split=0.1)




Train on 2085 samples, validate on 232 samples
Epoch 1/3





 736/2085 [=========>....................] - ETA: 15:17 - loss: 0.6490 - get_f1: 0.5992

In [ ]:
score = model.evaluate(X_test2, y_test2, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [28]:
pd.DataFrame(y_test2[:,:18]).to_csv('C:/Users/kfpj179/Desktop/Final Project/data/ActualLabels.csv', 
                                   index=False,header=True)
pd.DataFrame(model.predict(X_test2)).to_csv('C:/Users/kfpj179/Desktop/Final Project/data/PredictedLabels.csv', 
                                   index=False,header=True)

### 3.2 LSTM 2

In [1]:
deep_inputs = Input(shape=(2048,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(64, activation='sigmoid')(LSTM_Layer_1)
dense_layer_2 = Dense(18, activation='sigmoid')(dense_layer_1)
model2 = Model(inputs=deep_inputs, outputs=dense_layer_2)

NameError: name 'Input' is not defined

In [ ]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1])
history = model2.fit(X_train2, y_train2[:,:18], epochs=5, verbose=1, validation_split=0.1)

In [ ]:
score = model2.evaluate(X_test2, y_test2[:,:18], verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

### 3.3 LSTM 3 with 100 ICD labels

In [25]:
deep_inputs = Input(shape=(1280,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(256, activation='relu')(LSTM_Layer_1)
dense_layer_2 = Dense(100, activation='sigmoid')(dense_layer_1)
model3 = Model(inputs=deep_inputs, outputs=dense_layer_2)

In [26]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1])
history = model3.fit(X_train2, y_train2[:,18:], epochs=5, verbose=1, validation_split=0.1)

Train on 2430 samples, validate on 270 samples
Epoch 1/5
2430/2430 [==============================] - 237s 98ms/step - loss: 0.2533 - get_f1: 0.0225 - val_loss: 0.1976 - val_get_f1: 0.0000e+00
Epoch 2/5
2430/2430 [==============================] - 230s 94ms/step - loss: 0.1942 - get_f1: 0.0098 - val_loss: 0.1920 - val_get_f1: 0.0238
Epoch 3/5
2430/2430 [==============================] - 224s 92ms/step - loss: 0.1892 - get_f1: 0.0566 - val_loss: 0.1899 - val_get_f1: 0.0593
Epoch 4/5
2430/2430 [==============================] - 214s 88ms/step - loss: 0.1868 - get_f1: 0.0816 - val_loss: 0.1888 - val_get_f1: 0.0494
Epoch 5/5
2430/2430 [==============================] - 212s 87ms/step - loss: 0.1853 - get_f1: 0.0826 - val_loss: 0.1875 - val_get_f1: 0.0674


In [27]:
score = model3.evaluate(X_test2, y_test2[:,18:], verbose=1)

print("Test Score:", score[0])
print("Test F1 Accuracy:", score[1])

300/300 [==============================] - 10s 34ms/step
Test Score: 0.1898746101061503
Test F1 Accuracy: 0.07132874429225922


In [65]:
result = model2.predict(X_test2)
for i in result:
    i[i>=0.5] = 1
    i[i<0.5] = 0
np.max(result)

1.0

In [99]:
for i in range(0,0):
    print(i)

In [76]:
y_test2[:18]

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [57]:
pd.DataFrame(y_test2[:,18:])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.3 Add more samples by mixing up the sentences in X_train and creating duplicates

In [1]:
from PaddedClinicalBERTEmbeddings import PaddedClinicalBERTEmbeddings
embedding = PaddedClinicalBERTEmbeddings('negative for the following:  Fevers,\nchills, nausea, vomiting, night sweats, change in weight')

ModuleNotFoundError: No module named 'PaddedClinicalBERTEmbeddings'